In [1]:
import pandas as pd
import pandas_gbq
import plotly.express as px
import pandas_gbq
import sys, requests
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option("styler.format.thousands", ",")

In [2]:
#Change the date to the previous day
query = """
SELECT *, "Regular Season" as Category FROM `perceptive-ivy-290216.nba_api_box_defensive_player.all_time_reg_season`
UNION ALL
SELECT *, "Playoffs" as Category FROM `perceptive-ivy-290216.nba_api_box_defensive_player.all_time_playoffs`
"""
project_id = "perceptive-ivy-290216"
df_bq_misc_player = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [3]:
#USE FOR ALL TIME DATA
# df_bq3=df_bq_misc_player.copy()

#USE FOR SINGLE SEASON PLOTS
current_year="2023-24"
df_bq3=df_bq_misc_player[df_bq_misc_player["Season"]==current_year].copy()

df_bq3=df_bq3.fillna(0)
types="Regular Season"
# types="Playoffs"
df_bq2=df_bq3[df_bq3["Category"]==types]
df_bq2.head()

,GAME_ID,HOME_TEAM,AWAY_TEAM,TEAM,PLAYER_NAME,position,blocks,defensiveRebounds,matchupAssists,matchupFieldGoalPercentage,matchupFieldGoalsAttempted,matchupFieldGoalsMade,matchupMinutes,matchupThreePointerPercentage,matchupThreePointersAttempted,matchupThreePointersMade,matchupTurnovers,partialPossessions,playerPoints,steals,switchesOn,Season,Category
12311,0022301178,Minnesota Timberwolves,Atlanta Hawks,Minnesota Timberwolves,Jaden McDaniels,F,1,3,3,0.563,16,9,11:26,0.250,4,1,3,62.1,22,0,0,2023-24,Regular Season
12312,0022301178,Minnesota Timberwolves,Atlanta Hawks,Minnesota Timberwolves,Karl-Anthony Towns,F,1,4,4,0.467,15,7,10:33,0.286,7,2,1,55.1,19,0,0,2023-24,Regular Season
12313,0022301178,Minnesota Timberwolves,Atlanta Hawks,Minnesota Timberwolves,Rudy Gobert,C,5,15,3,0.407,27,11,14:29,0.250,4,1,1,76.0,25,1,0,2023-24,Regular Season
12314,0022301178,Minnesota Timberwolves,Atlanta Hawks,Minnesota Timberwolves,Anthony Edwards,G,0,4,2,0.571,7,4,10:39,0.000,3,0,1,55.0,9,2,0,2023-24,Regular Season
12315,0022301178,Minnesota Timberwolves,Atlanta Hawks,Minnesota Timberwolves,Mike Conley,G,0,5,5,0.571,7,4,11:38,0.000,1,0,1,63.6,11,2,0,2023-24,Regular Season


In [23]:
def team_color(team):
    if team =="Atlanta Hawks":
        return "#C8102E"
    elif team =="Dallas Mavericks":
        return '#0050B5'
    elif team =="Memphis Grizzlies":
        return '#0C2340'
    elif team =="Miami Heat":
        return '#862633'
    elif team =="Denver Nuggets":
        return '#0C2340'
    elif team =="Detroit Pistons":
        return '#1D4289'
    elif team =="Golden State Warriors":
        return '#1D4289'
    elif team =="Houston Rockets":
        return '#BA0C2F'
    elif team =="Indiana Pacers":
        return '#041E42'
    elif team =="Los Angeles Clippers":
        return '#0C2340'
    elif team =="Los Angeles Lakers":
        return '#6a00ed'
    elif team =="Portland Trail Blazers":
        return '#C8102E'
    elif team =="Phoenix Suns":
        return '#e56020'
    elif team =="Philadelphia 76ers":
        return '#1D4289'
    elif team =="Orlando Magic":
        return '#0057B7'
    elif team =="Oklahoma City Thunder":
        return '#0072CE'
    elif team =="New York Knicks":
        return '#FF8200'
    elif team =="New Orleans Pelicans":
        return '#0C2340'
    elif team =="Minnesota Timberwolves":
        return '#0C2340'
    elif team =="Milwaukee Bucks":
        return '#2C5234'
    elif team =="Brooklyn Nets":
        return '#707372'
    elif team =="Boston Celtics":
        return '#007A33'
    elif team =="Seattle SuperSonics":
        return '#00653A'
    elif team =="Charlotte Hornets":
        return '#00778B'
    elif team =="Buffalo Braves":
        return '#418FDE'
    elif team =="Chicago Bulls":
        return '#BA0C2F'       
    elif team =="Cleveland Cavaliers":
        return '#6F263D'
    elif team =="Washington Wizards":
        return '#C8102E'
    elif team =="Utah Jazz":
        return '#0C2340' 
    elif team =="Toronto Raptors":
        return '#C8102E'
    elif team =="San Antonio Spurs":
        return '#9EA2A2'
    elif team =="Sacramento Kings":
        return '#582C83'                  
    else:
        return '#3860C6'

df_bq2['color'] = df_bq2['TEAM'].apply(team_color)
player_color_dict=dict(zip(df_bq2['PLAYER_NAME'], df_bq2['color']))

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_16731/2750657442.py:69: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [28]:
#Get Totals For Each Category
df_totals=pd.DataFrame(df_bq2.groupby(by=['PLAYER_NAME','color'])[[
       'blocks', 'defensiveRebounds', 'matchupAssists', 'matchupFieldGoalsAttempted',
       'matchupFieldGoalsMade', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupTurnovers',
       'playerPoints', 'steals', 'switchesOn']].sum()).reset_index()

df_totals['matchupFieldGoalPercentage']=df_totals['matchupFieldGoalsMade']/df_totals['matchupFieldGoalsAttempted']
df_totals['matchupThreePointerPercentage']=df_totals['matchupThreePointersMade']/df_totals['matchupThreePointersAttempted']

#Get Game Counts for Each Player
df_count=pd.DataFrame(df_bq2.groupby(by=['PLAYER_NAME'])[["GAME_ID"]].count()).reset_index()
df_count.columns=["PLAYER_NAME","GP"]
df_totals=df_totals.merge(df_count, left_on=["PLAYER_NAME"],right_on=["PLAYER_NAME"])
df_totals.columns = df_totals.columns.str.replace('_', ' ') 
df_totals.head()

,PLAYER NAME,color,blocks,defensiveRebounds,matchupAssists,matchupFieldGoalsAttempted,matchupFieldGoalsMade,matchupThreePointersAttempted,matchupThreePointersMade,matchupTurnovers,playerPoints,steals,switchesOn,matchupFieldGoalPercentage,matchupThreePointerPercentage,GP
0,A.J. Lawson,#0050B5,3,36,24,107,56,23,9,12,139,10,0,0.523364,0.391304,42
1,AJ Green,#2C5234,4,55,66,230,108,78,29,33,283,9,0,0.469565,0.371795,56
2,AJ Griffin,#C8102E,2,16,20,66,32,24,3,7,71,1,0,0.484848,0.125,20
3,Aaron Gordon,#0C2340,45,294,251,867,376,286,96,112,951,55,0,0.433679,0.335664,72
4,Aaron Holiday,#BA0C2F,6,100,123,438,210,155,53,60,566,42,0,0.479452,0.341935,78


In [29]:
sort_category="matchupFieldGoalsMade"
# sort_category=[ 'blocks', 'defensiveRebounds', 'matchupAssists',
    #    'matchupFieldGoalPercentage', 'matchupFieldGoalsAttempted',
    #    'matchupFieldGoalsMade',
    #    'matchupThreePointerPercentage', 'matchupThreePointersAttempted',
    #    'matchupThreePointersMade', 'matchupTurnovers', 'partialPossessions',
    #    'playerPoints', 'steals', 'switchesOn']

In [30]:
#Sort by a category and then rank it. This will help get the Top Players in a category
df_totals['RK_Category']=df_totals[sort_category].rank(method="dense", ascending=False)
df_totals['RK_Category']=df_totals['RK_Category'].astype(int)
top=df_totals.sort_values(by=sort_category, ascending=False).head(10)
top

,PLAYER NAME,color,blocks,defensiveRebounds,matchupAssists,matchupFieldGoalsAttempted,matchupFieldGoalsMade,matchupThreePointersAttempted,matchupThreePointersMade,matchupTurnovers,playerPoints,steals,switchesOn,matchupFieldGoalPercentage,matchupThreePointerPercentage,GP,RK_Category
63,Brook Lopez,#2C5234,189,292,401,1756,839,397,151,150,1951,43,0,0.47779,0.380353,79,1
486,Nikola Jokic,#0C2340,66,744,417,1552,791,326,117,162,1806,105,0,0.509665,0.358896,78,2
169,Domantas Sabonis,#582C83,48,826,447,1407,685,311,122,198,1654,74,0,0.486851,0.392283,82,3
31,Anthony Davis,#6a00ed,176,712,426,1425,677,406,168,165,1645,89,0,0.475088,0.413793,75,4
290,Jarrett Allen,#6F263D,81,568,355,1298,636,280,93,174,1470,53,0,0.489985,0.332143,77,5
548,Rudy Gobert,#0C2340,162,697,367,1416,616,282,93,176,1449,52,0,0.435028,0.329787,76,6
97,Coby White,#BA0C2F,18,314,319,1210,615,524,199,139,1575,54,0,0.508264,0.379771,79,7
488,Nikola Vucevic,#BA0C2F,60,591,413,1229,606,330,114,171,1468,50,0,0.493084,0.345455,76,8
86,Chet Holmgren,#0072CE,190,519,334,1310,600,337,115,197,1480,53,0,0.458015,0.341246,82,9
153,Deni Avdija,#C8102E,36,456,296,1234,586,361,120,132,1417,60,0,0.474878,0.33241,75,10


In [21]:
#Get Player Names of the Top Category we have selected
names=top['PLAYER NAME'].to_list()

In [22]:
#Get the DF with Player Names from the top category
df_top=df_bq2[df_bq2['PLAYER_NAME'].isin(names)]
df_top.head()

,GAME_ID,HOME_TEAM,AWAY_TEAM,TEAM,PLAYER_NAME,position,blocks,defensiveRebounds,matchupAssists,matchupFieldGoalPercentage,matchupFieldGoalsAttempted,matchupFieldGoalsMade,matchupMinutes,matchupThreePointerPercentage,matchupThreePointersAttempted,matchupThreePointersMade,matchupTurnovers,partialPossessions,playerPoints,steals,switchesOn,Season,Category,color
12313,0022301178,Minnesota Timberwolves,Atlanta Hawks,Minnesota Timberwolves,Rudy Gobert,C,5,15,3,0.407,27,11,14:29,0.25,4,1,1,76.0,25,1,0,2023-24,Regular Season,#0C2340
12334,0022301194,Minnesota Timberwolves,Phoenix Suns,Minnesota Timberwolves,Rudy Gobert,C,1,4,11,0.455,11,5,14:33,0.00,3,0,3,63.8,11,2,0,2023-24,Regular Season,#0C2340
39574,0022300599,Minnesota Timberwolves,Oklahoma City Thunder,Minnesota Timberwolves,Rudy Gobert,C,0,14,0,0.000,0,0,12:07,0.00,0,0,0,63.4,0,0,0,2023-24,Regular Season,#0C2340
42018,0022300190,Milwaukee Bucks,Chicago Bulls,Chicago Bulls,Coby White,G,0,3,2,0.182,11,2,9:30,0.00,7,0,0,54.6,6,2,0,2023-24,Regular Season,#BA0C2F
42783,0022300506,Charlotte Hornets,Chicago Bulls,Chicago Bulls,Coby White,G,0,6,10,0.333,9,3,17:57,0.00,4,0,4,79.1,7,0,0,2023-24,Regular Season,#BA0C2F


In [18]:
#2 variable scatter plot

In [24]:
cat1="matchupFieldGoalsMade"
cat2="blocks"

fig_scatter = px.scatter(
  df_totals, 
  x=cat1, 
  y=cat2,
  color='PLAYER NAME',
  color_discrete_map=player_color_dict,
  title="<b>{} All Time Player {} vs. {}</b>".format(types,cat1,cat2),
  template="plotly_white",
  hover_data=['blocks', 'defensiveRebounds', 'matchupAssists',
       'matchupFieldGoalPercentage', 'matchupFieldGoalsAttempted',
       'matchupFieldGoalsMade',
       'matchupThreePointerPercentage', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupTurnovers',
       'playerPoints', 'steals', 'switchesOn'], 
  height=800, 
  width=1200,
  )

fig_scatter.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_scatter.update_traces(marker={'size': 10})

fig_scatter

In [25]:
fig_scatter.write_html("{} All Time Player {} vs. {}.html".format(types,cat1,cat2),full_html=False, include_plotlyjs='cdn')
# fig_scatter.write_html("Player {} vs. {} For The {} Season.html".format(cat1,cat2, current_year),full_html=False, include_plotlyjs='cdn')

In [ ]:
#Bar Chart For Category

In [27]:
fig_bar = px.bar(
  df_totals.sort_values(by=sort_category, ascending=False).head(25),
  x='PLAYER NAME', 
  y=sort_category,
  color='PLAYER NAME',
  color_discrete_map=player_color_dict,
  title="<b>{} All Time Player {}</b>".format(types,sort_category),
  template="plotly_white",
  hover_data=['blocks', 'defensiveRebounds', 'matchupAssists',
       'matchupFieldGoalPercentage', 'matchupFieldGoalsAttempted',
       'matchupFieldGoalsMade',
       'matchupThreePointerPercentage', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupTurnovers',
       'playerPoints', 'steals', 'switchesOn'], 
  height=800, 
  width=1200,
  )

fig_bar.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_bar

In [ ]:
fig_bar.write_html("{} All Time Player {}</b>".format(types,sort_category),full_html=False, include_plotlyjs='cdn')
# fig_bar.write_html("Player {} For The {} {}</b>".format(sort_category, current_year, types),full_html=False, include_plotlyjs='cdn')

In [ ]:
#MULTI METRIC PLOTS TOGETHER

In [43]:
sort_categories=[ 'blocks', 'defensiveRebounds', 'matchupAssists', 'matchupFieldGoalsAttempted',
       'matchupFieldGoalsMade', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupTurnovers',
       'playerPoints', 'steals', 'switchesOn']

In [50]:
for i in sort_categories:
    df_totals['RK_Category']=df_totals[i].rank(method="dense", ascending=False)
    df_totals=df_totals.fillna(0)
    df_totals['RK_Category']=df_totals['RK_Category'].astype(int)
    top=df_totals.sort_values(by=i, ascending=False).head(10)
    names=top['PLAYER NAME'].to_list()
    df_top=df_bq2[df_bq2['PLAYER_NAME'].isin(names)]

    fig_bar_multi = px.bar(
    df_totals.sort_values(by=i, ascending=False).head(25),
    x='PLAYER NAME', 
    y=i,
    color='PLAYER NAME',
    color_discrete_map=player_color_dict,
    title="Player {} For The {} {}</b>".format(i, current_year, types),
    template="plotly_white",
  hover_data=['blocks', 'defensiveRebounds', 'matchupAssists',
       'matchupFieldGoalPercentage', 'matchupFieldGoalsAttempted',
       'matchupFieldGoalsMade',
       'matchupThreePointerPercentage', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupTurnovers',
       'playerPoints', 'steals', 'switchesOn'], 
    height=800, 
    width=1200,
    )

    fig_bar_multi.update_layout(
        title_x=0.5,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=20)),
        xaxis = dict(tickfont = dict(size=20)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    # fig_bar_multi.update_traces(marker_color='#0047ab')
    # fig_bar_multi.write_html("{} All Time Player {}.html".format(types,i),full_html=False, include_plotlyjs='cdn')
    fig_bar_multi.write_html("Player {} For The {} {}.html".format(i, current_year, types),full_html=False, include_plotlyjs='cdn')

In [48]:
sort_categories=['matchupFieldGoalPercentage','matchupThreePointerPercentage']

In [49]:
for i in sort_categories:
    df_totals['RK_Category']=df_totals[i].rank(method="dense", ascending=False)
    df_totals=df_totals.fillna(0)
    df_totals['RK_Category']=df_totals['RK_Category'].astype(int)
    top=df_totals.sort_values(by=i, ascending=False).head(10)
    names=top['PLAYER NAME'].to_list()
    df_top=df_bq2[df_bq2['PLAYER_NAME'].isin(names)]

    fig_bar_multi = px.bar(
    df_totals[df_totals["matchupFieldGoalsAttempted"]>100].sort_values(by=i, ascending=False).head(25),
    x='PLAYER NAME', 
    y=i,
    color='PLAYER NAME',
    color_discrete_map=player_color_dict,
    title="Player {} For The {} {}</b>".format(i, current_year, types),
    template="plotly_white",
  hover_data=['blocks', 'defensiveRebounds', 'matchupAssists',
       'matchupFieldGoalPercentage', 'matchupFieldGoalsAttempted',
       'matchupFieldGoalsMade',
       'matchupThreePointerPercentage', 'matchupThreePointersAttempted',
       'matchupThreePointersMade', 'matchupTurnovers',
       'playerPoints', 'steals', 'switchesOn'], 
    height=800, 
    width=1200,
    )

    fig_bar_multi.update_layout(
        title_x=0.5,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=20)),
        xaxis = dict(tickfont = dict(size=20)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    # fig_bar_multi.update_traces(marker_color='#0047ab')
    # fig_bar_multi.write_html("{} All Time Player {}.html".format(types,i),full_html=False, include_plotlyjs='cdn')
    fig_bar_multi.write_html("Player {} For The {} {}.html".format(i, current_year, types),full_html=False, include_plotlyjs='cdn')